In [2]:
import pandas as pd
products = pd.read_csv('data_skinrec_bismillah.csv', sep=';')
products.head()

,product_href,product_name,product_type,brand,price,description,picture_src,sensitive,combination,oily,...,Soothing,UV-Protection,Moisturizing,Acne-Free,Refreshing,Pore-Care,Black-Spot,Brightening,No-Whitecast,Hydrating
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,209000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,0,0,1,...,0,0,0,1,0,1,0,1,0,0
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,181800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,0,1,0,...,1,0,0,0,0,0,0,0,0,0
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,149000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",1,1,1,...,1,0,0,0,0,0,0,0,0,0
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,290000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,0,0,1,...,0,0,0,1,0,1,0,1,0,0
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,194650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",0,0,0,...,1,0,0,0,0,0,0,1,0,0


In [20]:
def recommend_skincare(product_type, skin_types, skin_problems, notable_effects):
    # Filter produk berdasarkan product_type
    filtered_df = products[products['product_type'] == product_type]
    
    # Filter produk berdasarkan skin types yang sesuai (nilai 1 hanya untuk skin_types yang dipilih)
    all_skin_types = ['sensitive', 'combination', 'dry', 'oily', 'normal']
    other_skin_types = [stype for stype in all_skin_types if stype not in skin_types]
    
    # Produk harus memiliki nilai 1 untuk semua skin_types yang dipilih dan 0 untuk lainnya
    skin_type_mask = (filtered_df[skin_types].sum(axis=1) == len(skin_types)) & (filtered_df[other_skin_types].sum(axis=1) == 0)
    filtered_df = filtered_df[skin_type_mask]
    
    # Hitung jumlah 1 di kolom skin problems yang sesuai dengan pilihan user
    total_skin_problems = filtered_df[['kulit kusam', 'jerawat', 'bekas jerawat', 'pori-pori besar', 'flek hitam',
                                       'garis halus dan kerutan', 'komedo', 'warna kulit tidak merata', 'kemerahan',
                                       'kulit kendur']].sum(axis=1)
    skin_problems_match = filtered_df[skin_problems].sum(axis=1) / total_skin_problems
    
    # Hitung jumlah 1 di kolom notable effects yang sesuai dengan pilihan user
    total_notable_effects = filtered_df[['Acne-Free', 'Soothing', 'Brightening', 'Moisturizing', 'Hydrating',
                                         'Pore-Care', 'Anti-Aging', 'Balancing', 'UV-Protection', 'Skin-Barrier',
                                         'Refreshing', 'Oil-Control', 'No-Whitecast', 'Black-Spot']].sum(axis=1)
    notable_effects_match = filtered_df[notable_effects].sum(axis=1) / total_notable_effects
    
    # Gabungkan semua skor kesesuaian
    total_score = (skin_problems_match + notable_effects_match)/2
    
    # Urutkan produk berdasarkan total_score secara descending
    recommended_products = filtered_df.assign(total_score=total_score).sort_values(by='total_score', ascending=False)
    recommended_products = recommended_products[recommended_products['total_score'] > 0]
    
    return recommended_products[['product_name', 'brand', 'price', 'description', 'picture_src', 'total_score', 'rating', 'product_type']].head(5)

# Contoh penggunaan fungsi recommend_skincare dengan multiple input
skin_types = ['oily']
skin_problems = ['jerawat', 'bekas jerawat']
notable_effects = ['Acne-Free']

recommended_products = recommend_skincare('Face Wash', skin_types, skin_problems, notable_effects)
recommended_products

,product_name,brand,price,description,picture_src,total_score,rating,product_type
772,POND's Acne Solution Facial Foam,POND'S,19285,Pond's Acne Solution Facial Foam pertama yang ...,https://www.beautyhaul.com/assets/uploads/prod...,0.750000,5,Face Wash
713,NAUFA Face Soap 50 gr,NAUFA,20900,"Alcohol Free, based on customer's review, this...",https://www.beautyhaul.com/assets/uploads/prod...,0.666667,5,Face Wash
0,ACWELL Bubble Free PH Balancing Cleanser,ACWELL,209000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,0.625000,5,Face Wash
1116,Wardah Acnederm Pure Foaming Cleanser,WARDAH,24500,A special formulated facial cleanser to keep y...,https://wardah-mainsite.s3-ap-southeast-1.amaz...,0.250000,5,Face Wash
385,Garnier Bright Complete 3 in 1 Anti Acne Face ...,GARNIER,35000,Baru! Pembersih wajah Anti Acne Pertama* denga...,http://image.femaledaily.com/dyn/500/images/pr...,0.250000,5,Face Wash
